# LDA Topic Modeling with Gensim

Heavily modified and expanded based upon Gensim tutorial

By Jon Chun
28 Mar 2022 Updated

# Setup and Configuration

In [ ]:
%whos

Interactive namespace is empty.


In [ ]:
%%time

# NOTE: 54s

!pip install pyldavis

CPU times: user 59.5 ms, sys: 7.47 ms, total: 66.9 ms
Wall time: 8.58 s


In [ ]:
%%time

# NOTE: 9s

!pip install contractions

CPU times: user 56.1 ms, sys: 3.51 ms, total: 59.6 ms
Wall time: 8 s


In [ ]:
!pip install gensim

In [ ]:
import numpy
import gensim
print("numpy version:", numpy.__version__)
print("gensim version:", gensim.__version__)


numpy version: 1.26.4
gensim version: 4.3.3


In [ ]:
# Installation cell: Run this in a separate cell and then restart the runtime
%pip uninstall -y numpy
%pip install numpy==1.26.4
%pip install gensim==4.3.3


Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


# **[RESTART RUNTIME]**

In [ ]:
import numpy
import gensim
print("numpy version:", numpy.__version__)
print("gensim version:", gensim.__version__)


numpy version: 1.26.4
gensim version: 4.3.3


## Configure Jupyter Notebook

In [ ]:
%%time

# NOTE:

## Configure Jupyter Notebook

# Ignore warnings

import warnings
warnings.filterwarnings('ignore')

# Enable multiple outputs from one code cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.display import display
from IPython.display import Image
from ipywidgets import widgets, interactive

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

CPU times: user 134 ms, sys: 11 ms, total: 145 ms
Wall time: 167 ms


## Import Libraries

In [ ]:
%%time

# NOTE:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

CPU times: user 985 ms, sys: 85.4 ms, total: 1.07 s
Wall time: 1.54 s


# Data

**[STEP 1]** Find a mid-length+ novel you are familiar with and download the *.txt version to your local harddrive:
1. https://gutenberg.net.au/ (less restrictive IP)
2. https://www.gutenberg.org/ebooks/

**[STEP 2]** download and save the corpus using a simple filename like: "book_ulysses_james-joyce.txt"
1. all lowercase
2. only text or '_' or '-' (no other punctuation or whitespaces)
3. no leading numbers
these are OS-friendly and legal Python variable names

**[STEP 3]** open the text file and trim/delete the header and footer cruft (not core novel)

**[STEP 4]** THEN, upload this file in the next code cell


In [ ]:
# prompt: upload a file and save the root of the filename to textfile_name

from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  textfile_name = fn.split('.')[0]


Saving 2010s.txt to 2010s.txt
User uploaded file "2010s.txt" with length 2412416 bytes


In [ ]:
# Multiple uploads of same file creates multiple versions with " (n}.txt" suffix
# Must be deleted to start over

# !rm book_*

In [ ]:
# First manually upload your text file via the Jupyter menu
#   then run this code cell to make sure it appears here in the working directory

!ls

2010s.txt  sample_data



# LDA Model

Introduces Gensim's LDA model and demonstrates its use on the NIPS corpus.


The purpose of this tutorial is to demonstrate how to train and tune an LDA model.

In this tutorial we will:

* Load input data.
* Pre-process that data.
* Transform documents into bag-of-words vectors.
* Train an LDA model.

This tutorial will **not**:

* Explain how Latent Dirichlet Allocation works
* Explain how the LDA model performs inference
* Teach you all the parameters and options for Gensim's LDA implementation

If you are not familiar with the LDA model or how to use it in Gensim, I (Olavur Mortensen)
suggest you read up on that before continuing with this tutorial. Basic
understanding of the LDA model should suffice. Examples:

* `Introduction to Latent Dirichlet Allocation <http://blog.echen.me/2011/08/22/introduction-to-latent-dirichlet-allocation>`_
* Gensim tutorial: `sphx_glr_auto_examples_core_run_topics_and_transformations.py`
* Gensim's LDA model API docs: :py:class:`gensim.models.LdaModel`

I would also encourage you to consider each step when applying the model to
your data, instead of just blindly applying my solution. The different steps
will depend on your data and possibly your goal with the model.





## Read Textfile

In [ ]:
!ls -al

total 2372
drwxr-xr-x 1 root root    4096 Jun  5 19:10 .
drwxr-xr-x 1 root root    4096 Jun  5 19:01 ..
-rw-r--r-- 1 root root 2412416 Jun  5 19:10 2010s.txt
drwxr-xr-x 4 root root    4096 Jun  4 21:29 .config
drwxr-xr-x 1 root root    4096 Jun  4 21:29 sample_data


In [ ]:
with open(f"{textfile_name}.txt", 'r', encoding='ascii', errors='ignore') as f:
    book_all_str = f.read()

type(book_all_str)

str

In [ ]:
# Get char count
len(book_all_str)

2358460

In [ ]:
print(book_all_str[:1000])

 RELLESTSEB1 #E H T
B NADROJ
NOSRETEP
SELUR2 1
E F I LR O F
SOAHC OT ETODITNA NA
no egreme ot srekniht tnatropmi tsom eht fo enO"
ROTATCEPSEHT ".sraey ynam rof egats dlrow eht
EGDIOD NAMRON YB DROWEROF

 2

Jordan B. Peterson 12 RULES FOR LIFE
An Antidote for Chaos
Foreword by Norman Doidge Illustrations by Ethan Van Scriver
   3

Table of Contents
Foreword by Norman Doidge Overture
RULE 1 / Stand up straight with your shoulders back
RULE 2 / Treat yourself like someone you are responsible for helping
RULE 3 / Make friends with people who want the best for you
RULE 4 / Compare yourself to who you were yesterday, not to who someone else is today RULE 5 / Do not let your children do anything that makes you dislike them RULE 6 / Set your house in perfect order before you criticize the world
RULE 7 / Pursue what is meaningful (not what is expedient)
RULE 8 / Tell the truthor, at least, dont lie
RULE 9 / Assume that the person you are listening to might know something you dont RULE 10 / Be 

## Split the Book/Corpus into Paragraphs/Documents

In [ ]:
# Split into paragraphs

# CUSTOMIZE: Might have to change paragraphs separator either
# (a) '\n\n'
# (b) '\n'
# (c) something else (e.g. no '\n' in text)

parags_ls = book_all_str.split('\n') # MIGHT HAVE TO CHANGE '\n' OR '\n\n' (two newlines)

# Replace stray/embedded /n with a space
book_parags_clean_ls = []
for aparag in parags_ls:
  book_parags_clean_ls.append(aparag.replace('\n',' '))

parags_ls = book_parags_clean_ls
parags_ls[:50]

[' RELLESTSEB1 #E H T',
 'B NADROJ',
 'NOSRETEP',
 'SELUR2 1',
 'E F I LR O F',
 'SOAHC OT ETODITNA NA',
 'no egreme ot srekniht tnatropmi tsom eht fo enO"',
 'ROTATCEPSEHT ".sraey ynam rof egats dlrow eht',
 'EGDIOD NAMRON YB DROWEROF',
 '',
 ' 2',
 '',
 'Jordan B. Peterson 12 RULES FOR LIFE',
 'An Antidote for Chaos',
 'Foreword by Norman Doidge Illustrations by Ethan Van Scriver',
 '   3',
 '',
 'Table of Contents',
 'Foreword by Norman Doidge Overture',
 'RULE 1 / Stand up straight with your shoulders back',
 'RULE 2 / Treat yourself like someone you are responsible for helping',
 'RULE 3 / Make friends with people who want the best for you',
 'RULE 4 / Compare yourself to who you were yesterday, not to who someone else is today RULE 5 / Do not let your children do anything that makes you dislike them RULE 6 / Set your house in perfect order before you criticize the world',
 'RULE 7 / Pursue what is meaningful (not what is expedient)',
 'RULE 8 / Tell the truthor, at least, dont li

In [ ]:
# Get paragraph count

# VERIFY: Should not be 0, 1 or <100, <1000 depending on task/text

len(parags_ls)

10373

In [ ]:
# VERIFY: first paragraph (after title and chapter headings)

print(parags_ls[1000:1010])
print('\n')
# Paragraph char count
len(parags_ls[3])

['grab while the grabbings good. But now he wont be able to extract his fist, now full of treats, from the too-narrow opening of the jar. Not without unclenching his hand. Not without relinquishing what he already has. And thats just what he wont do. The monkey-catcher can just walk over to the jar and pick up the monkey. The animal will not sacrifice the part to preserve the whole.', 'Something valuable, given up, ensures future prosperity. Something valuable, sacrificed, pleases the Lord. What is most valuable, and best sacrificed?or, what is at least emblematic of that? A choice cut of meat. The best animal in a flock. A most valued possession. Whats above even that? Something intensely personal and painful to give up. Thats symbolized, perhaps, in Gods insistence on circumcision as part of Abrahams sacrificial routine, where the part is offered, symbolically, to redeem the whole. Whats beyond that? What pertains more closely to the whole person, rather than the part? What constitut

8

In [ ]:
# Delete any paragraphs shorter than MIN_LEN_PARAG

# CUSTOMIZE: Set  minimum paragraph length MIN_LEN_PARAG
MIN_LEN_PARAG = 5
MIN_LEN_DOC = 1000

# Delete any paragraphs shorter than MIN_LEN_PARAG
parags_ls = [x for x in parags_ls if len(x) > MIN_LEN_PARAG]

# Trim any leading/trailing/multiple embedded whitespaces
parags_ls = [' '.join(x.split()) for x in parags_ls]

len(parags_ls)

8472

In [ ]:
start_indx = 1000
end_indx = 1010
for indx, aparag in enumerate(parags_ls[start_indx:end_indx]):
  print(f"Paragraph #{start_indx+indx}: {aparag}")

Paragraph #1000: At about the same time I was seeing this client, the media was all afire with stories of recovered memoriesparticularly of sexual assault. The dispute raged apace: were these genuine accounts of past trauma? Or were they post-hoc constructs, dreamed up as a consequence of pressure wittingly or unwittingly applied by incautious therapists, grasped onto desperately by clinical clients all-too-eager to find a simple cause for all their
Paragraph #1001: trouble? Sometimes, it was the former, perhaps; and sometimes the latter. I understood much more clearly and precisely, however, how easy it might be to instill a false memory into the mental landscape as soon as my client revealed her uncertainty about her sexual experiences. The past appears fixed, but its notnot in an important psychological sense. There is an awful lot to the past, after all, and the way we organize it can be subject to drastic revision.
Paragraph #1002: Imagine, for example, a movie where nothing but t

## Customize Stopwords

In [ ]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

stopwords_ls = stopwords.words('english')

print(f'\nThe first ten stopwords:')
stopwords_ls[20:]
print(f'\n\nThere are [{len(stopwords_ls)}] English stopwords imported from NLTK')

# CUSTOMIZE: Optionally add low-info/semantically noisy words you want to remove from the corpus
stopwords_custom_ls = ['chapter', 'bazinga', 'he']

stopwords_ls = stopwords_ls + stopwords_custom_ls
stopwords_ls.sort()
stopwords_ls[-10:]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True


The first ten stopwords:


['before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 "he'd",
 "he'll",
 'her',
 'here',
 'hers',
 'herself',
 "he's",
 'him',
 'himself',
 'his',
 'how',
 'i',
 "i'd",
 'if',
 "i'll",
 "i'm",
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it'd",
 "it'll",
 "it's",
 'its',
 'itself',
 "i've",
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 're',
 's',
 'same',
 'shan',
 "shan't",
 'she',
 "she'd",
 "she'll",
 "she's",
 'should',
 'sh



There are [198] English stopwords imported from NLTK


['y',
 'you',
 "you'd",
 "you'll",
 "you're",
 "you've",
 'your',
 'yours',
 'yourself',
 'yourselves']

In [ ]:
# Remove all stopwords before expanding contractions

parags_nostop_ls_ls = [[atoken for atoken in aparag.split() if not atoken in stopwords_ls] for aparag in parags_ls]

# VERIFY: Pick a paragraph number to test

parag_no = 1000

print(f'BEFORE: {parags_ls[parag_no]} AFTER: {parags_nostop_ls_ls[parag_no]}')
print(f'BEFORE: {len(parags_ls[parag_no].split())} AFTER: {len(parags_nostop_ls_ls[parag_no])}')

print(f'BEFORE: {len(parags_ls)} AFTER: {len(parags_nostop_ls_ls)}')

# Delete empty
parags_nostop_ls_ls = [x for x in parags_nostop_ls_ls if len(x) > 0]

print(f'BEFORE: {len(parags_ls)} AFTER: {len(parags_nostop_ls_ls)}')

BEFORE: At about the same time I was seeing this client, the media was all afire with stories of recovered memoriesparticularly of sexual assault. The dispute raged apace: were these genuine accounts of past trauma? Or were they post-hoc constructs, dreamed up as a consequence of pressure wittingly or unwittingly applied by incautious therapists, grasped onto desperately by clinical clients all-too-eager to find a simple cause for all their AFTER: ['At', 'time', 'I', 'seeing', 'client,', 'media', 'afire', 'stories', 'recovered', 'memoriesparticularly', 'sexual', 'assault.', 'The', 'dispute', 'raged', 'apace:', 'genuine', 'accounts', 'past', 'trauma?', 'Or', 'post-hoc', 'constructs,', 'dreamed', 'consequence', 'pressure', 'wittingly', 'unwittingly', 'applied', 'incautious', 'therapists,', 'grasped', 'onto', 'desperately', 'clinical', 'clients', 'all-too-eager', 'find', 'simple', 'cause']
BEFORE: 68 AFTER: 40
BEFORE: 8472 AFTER: 8472
BEFORE: 8472 AFTER: 8441


In [ ]:
parags_nostop_ls_ls[1000:1100]

[['At',
  'time',
  'I',
  'seeing',
  'client,',
  'media',
  'afire',
  'stories',
  'recovered',
  'memoriesparticularly',
  'sexual',
  'assault.',
  'The',
  'dispute',
  'raged',
  'apace:',
  'genuine',
  'accounts',
  'past',
  'trauma?',
  'Or',
  'post-hoc',
  'constructs,',
  'dreamed',
  'consequence',
  'pressure',
  'wittingly',
  'unwittingly',
  'applied',
  'incautious',
  'therapists,',
  'grasped',
  'onto',
  'desperately',
  'clinical',
  'clients',
  'all-too-eager',
  'find',
  'simple',
  'cause'],
 ['trouble?',
  'Sometimes,',
  'former,',
  'perhaps;',
  'sometimes',
  'latter.',
  'I',
  'understood',
  'much',
  'clearly',
  'precisely,',
  'however,',
  'easy',
  'might',
  'instill',
  'false',
  'memory',
  'mental',
  'landscape',
  'soon',
  'client',
  'revealed',
  'uncertainty',
  'sexual',
  'experiences.',
  'The',
  'past',
  'appears',
  'fixed,',
  'notnot',
  'important',
  'psychological',
  'sense.',
  'There',
  'awful',
  'lot',
  'past,',


**[CUSTOMIZE] Stopwords list by adding or deleting tokens**

## Expand Contractions

In [ ]:
# Expand Contractions (e.g. can't -> can not)

import contractions

# Test
print('Not fixed:')
contractions.fix("yall're happy now", slang=False) # default: true
print('\n\nFixed:')
contractions.fix("you're happy now", slang=False) # default: true

Not fixed:


"yall're happy now"



Fixed:


'you are happy now'

In [ ]:
# Expand all Contractions paragraph by paragraph
parags_nostop_expand_ls_ls = [[contractions.fix(atoken) for atoken in aparag_nostop_ls] for aparag_nostop_ls in parags_nostop_ls_ls]

# VERIFY: set index range
parags_nostop_expand_ls_ls[1000:1100]


[['At',
  'time',
  'I',
  'seeing',
  'client,',
  'media',
  'afire',
  'stories',
  'recovered',
  'memoriesparticularly',
  'sexual',
  'assault.',
  'The',
  'dispute',
  'raged',
  'apace:',
  'genuine',
  'accounts',
  'past',
  'trauma?',
  'Or',
  'post-hoc',
  'constructs,',
  'dreamed',
  'consequence',
  'pressure',
  'wittingly',
  'unwittingly',
  'applied',
  'incautious',
  'therapists,',
  'grasped',
  'onto',
  'desperately',
  'clinical',
  'clients',
  'all-too-eager',
  'find',
  'simple',
  'because'],
 ['trouble?',
  'Sometimes,',
  'former,',
  'perhaps;',
  'sometimes',
  'latter.',
  'I',
  'understood',
  'much',
  'clearly',
  'precisely,',
  'however,',
  'easy',
  'might',
  'instill',
  'false',
  'memory',
  'mental',
  'landscape',
  'soon',
  'client',
  'revealed',
  'uncertainty',
  'sexual',
  'experiences.',
  'The',
  'past',
  'appears',
  'fixed,',
  'notnot',
  'important',
  'psychological',
  'sense.',
  'There',
  'awful',
  'lot',
  'past,'

In [ ]:
len(parags_nostop_expand_ls_ls)
parags_nostop_expand_ls_ls[1000]

8441

['At',
 'time',
 'I',
 'seeing',
 'client,',
 'media',
 'afire',
 'stories',
 'recovered',
 'memoriesparticularly',
 'sexual',
 'assault.',
 'The',
 'dispute',
 'raged',
 'apace:',
 'genuine',
 'accounts',
 'past',
 'trauma?',
 'Or',
 'post-hoc',
 'constructs,',
 'dreamed',
 'consequence',
 'pressure',
 'wittingly',
 'unwittingly',
 'applied',
 'incautious',
 'therapists,',
 'grasped',
 'onto',
 'desperately',
 'clinical',
 'clients',
 'all-too-eager',
 'find',
 'simple',
 'because']

In [ ]:
# OPTIONAL: Remove stopwords after contraction expansion

## Clean Text

In [ ]:
import string

# Strip leading/trailing whitespace
parags_nostop_expand_ls_ls = [[word.strip() for word in sublst] for sublst in parags_nostop_expand_ls_ls]

# Strip leading/trailing punctuation
parags_nostop_expand_ls_ls = [[word.strip(string.punctuation) for word in sublst] for sublst in parags_nostop_expand_ls_ls]

# Lower case
parags_nostop_expand_ls_ls = [[word.lower() for word in sublst] for sublst in parags_nostop_expand_ls_ls]

# VERIFY
print(parags_nostop_expand_ls_ls[100])


['when', 'i', 'taking', 'break', 'or', 'avoiding', 'work', 'i', 'often', 'turned', 'quora', 'looking']


## [SKIP] Tokenize Text

In [ ]:
"""
from nltk.tokenize import RegexpTokenizer
dir(RegexpTokenizer)
""";

In [ ]:
"""
# Tokenize the documents.
from nltk.tokenize import RegexpTokenizer

# Split the documents into tokens.
tokenizer = RegexpTokenizer(r'\w+')

book_parags_ls_ls = []
# Split the documents into tokens.
for aparag in book_parags_ls:
  aparag_str = aparag.lower()
  aparag_ls = tokenizer.tokenize(aparag_str)
  book_parags_ls_ls.append(aparag_ls)

# Remove numbers, but not words that contain numbers.
book_parags_ls_ls = [[token for token in parag if not token.isnumeric()] for parag in book_parags_ls_ls]

# Remove words that are only one character.
book_parags_ls_ls = [[token for token in parag if len(token)>1] for parag in book_parags_ls_ls]

print(book_parags_ls_ls[0])
""";

In [ ]:
"""
# Verify docs

# content and token count of first doc
book_parags_ls_ls[0]
print(f'\n\nThere are {len(book_parags_ls_ls[0])} tokens in the first document')
""";

In [ ]:
# book_parags_ls_ls[:10]

In [ ]:
# book_parags_ls_ls[1000:1100]

## Agglomerate paragraphs into Documents

In [ ]:
print(parags_nostop_expand_ls_ls[0])

['rellestseb1', 'e', 'h', 't']


In [ ]:
len(parags_nostop_expand_ls_ls)

8441

In [ ]:
# CUSTOMIZE: Set minimum Document length in terms of tokens

MIN_LEN_DOC = 250
# MIN_LEN_DOC

In [ ]:
# Agglomerate paragraphs into Documents of MIN_LEN_DOC=1000 chars

parag_ct = len(parags_nostop_expand_ls_ls)

# doc_now_str = ''
parag_now_ls = []
docs_ls = []

for i in range(parag_ct):
  # print(f'Processing Paragraph #{i}')
  if len(parags_nostop_expand_ls_ls[i]) < 1:
    continue
  else:
    # parag_now_ls.append(parags_nostop_expand_ls_ls[i])
    # doc_now_str += parag_now_str
    # parag_now_len += len(parag_now_ls)
    if len(parag_now_ls) > MIN_LEN_DOC:
      docs_ls.append(parag_now_ls)
      parag_now_ls = []
    else:
      parag_now_ls.extend(parags_nostop_expand_ls_ls[i])

if len(parag_now_ls) > 0:
  docs_ls[-1] = docs_ls[-1] + parag_now_ls

print(f'There are now {len(docs_ls)} Documents of {MIN_LEN_DOC} chars or more')

There are now 748 Documents of 250 chars or more


In [ ]:
# Remove docs that are empty

docs_ls = [adoc for adoc in docs_ls if len(adoc)>0]
print(f'There are now {len(docs_ls)} non-empty Documents of {MIN_LEN_DOC}')

There are now 748 non-empty Documents of 250


In [ ]:
# import nltk
# nltk.download('punkt')

In [ ]:
"""
book_parags_nostop_noempty_ls = []
for i,para in enumerate(book_parags_nostop_noempty_ls_ls):
  print(f'Processing line #{i}: {para}')
  para_str = ' '.join(para)
  sentences = nltk.sent_tokenize(para_str)
  book_parags_nostop_noempty_ls.extend(sentences)
book_parags_nostop_noempty_ls[:5]
""";

In [ ]:
# VERFIY: the first 5 docs

len(docs_ls[0])
print(' '.join(docs_ls[0]))
print('\n')
print(' '.join(docs_ls[1]))

254

rellestseb1 e h t b nadroj nosretep selur2 1 e f i lr o f soahc ot etoditna na egreme ot srekniht tnatropmi tsom eht fo eno rotatcepseht sraey ynam rof egats dlrow eht egdiod namron yb drowerof jordan b peterson 12 rules for life an antidote chaos foreword norman doidge illustrations ethan van scriver table contents foreword norman doidge overture rule 1  stand straight shoulders back rule 2  treat like someone responsible helping rule 3  make friends people want best rule 4  compare yesterday someone else today rule 5  do let children anything makes dislike rule 6  set house perfect order criticize world rule 7  pursue meaningful not expedient rule 8  tell truthor least do not lie rule 9  assume person listening might know something do not rule 10  be precise speech rule 11  do bother children skateboarding rule 12  pet cat encounter one street coda endnotes acknowledgements follow penguin 4 foreword rules more rules really is not life complicated enough restricting enough without abs

In [ ]:
# prompt: wordwrap print ' '.join(docs_ls[0]))

import textwrap

print(textwrap.fill(' '.join(docs_ls[0])))


rellestseb1 e h t b nadroj nosretep selur2 1 e f i lr o f soahc ot
etoditna na egreme ot srekniht tnatropmi tsom eht fo eno rotatcepseht
sraey ynam rof egats dlrow eht egdiod namron yb drowerof jordan b
peterson 12 rules for life an antidote chaos foreword norman doidge
illustrations ethan van scriver table contents foreword norman doidge
overture rule 1  stand straight shoulders back rule 2  treat like
someone responsible helping rule 3  make friends people want best rule
4  compare yesterday someone else today rule 5  do let children
anything makes dislike rule 6  set house perfect order criticize world
rule 7  pursue meaningful not expedient rule 8  tell truthor least do
not lie rule 9  assume person listening might know something do not
rule 10  be precise speech rule 11  do bother children skateboarding
rule 12  pet cat encounter one street coda endnotes acknowledgements
follow penguin 4 foreword rules more rules really is not life
complicated enough restricting enough without abs

In [ ]:
# VERIFY: the first 20 tokens in the 50th doc

docs_ls[49][:20]

['it',
 'true',
 'idea',
 'virtuous',
 'self-sacrifice',
 'deeply',
 'embedded',
 'western',
 'culture',
 'at',
 'least',
 'insofar',
 'west',
 'influenced',
 'christianity',
 'based',
 'imitation',
 'someone',
 'performed',
 'ultimate']

In [ ]:
# VERIFY: the last 100 tokens in the last doc

docs_ls[-1][-100:]

['82',
 'zogby',
 'international',
 '190',
 'about',
 'the',
 'author',
 'bren',
 'brown',
 'ph.d',
 'lmsw',
 'research',
 'professor',
 'university',
 'houston',
 'graduate',
 'college',
 'social',
 'work',
 'she',
 'nationally',
 'renowned',
 'speaker',
 'numerous',
 'teaching',
 'awards',
 'including',
 'colleges',
 'outstanding',
 'faculty',
 'award',
 'her',
 'groundbreaking',
 'research',
 'featured',
 'pbs',
 'npr',
 'cnn',
 'brens',
 '2010',
 'tedxhouston',
 'talk',
 'the',
 'power',
 'vulnerability',
 'one',
 'top',
 'ten',
 'viewed',
 'ted',
 'talks',
 'ted.com',
 'approximately',
 '5',
 'million',
 'viewers',
 'additionally',
 'bren',
 'gave',
 'closing',
 'talk',
 '2012',
 'ted',
 'conference',
 'talked',
 'shame',
 'courage',
 'innovation',
 'she',
 'also',
 'author',
 'the',
 'gifts',
 'imperfection',
 '2010',
 'i',
 'thought',
 'it',
 'was',
 'just',
 'me',
 '2007',
 'connections',
 '2009',
 'shame-resilience',
 'curriculum',
 'facilitated',
 'helping',
 'professionals',

## Pre-process and vectorize the documents

As part of preprocessing, we will:

* Expand Contractions
* Tokenize (split the documents into tokens)
* Define stopwords and filter out
* Lemmatize the tokens.
* Compute bigrams.
* Compute a bag-of-words representation of the data.

First we tokenize the text using a regular expression tokenizer from NLTK. We
remove numeric tokens and tokens that are only a single character, as they
don't tend to be useful, and the dataset contains a lot of them.

.. Important::

   This tutorial uses the nltk library for preprocessing, although you can
   replace it with something else if you want.




### Lemmatize

We use the WordNet lemmatizer from NLTK. A lemmatizer is preferred over a
stemmer in this case because it produces more readable words. Output that is
easy to read is very desirable in topic modelling.




In [ ]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


True

In [ ]:
%%time

# NOTE: 1m25s @10:14 on 20230401 Colab Pro (JourneyToTheWest_4vol_2003_Chengen)

# Lemmatize the documents

from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet

def get_wordnet_pos(word):
    """Map POS tag to first character used by WordNetLemmatizer"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()
# NOTE: Must provide lemmatizer.lemmatize(token, POS) with POS, not just token
docs_lemma_ls = [[lemmatizer.lemmatize(token, get_wordnet_pos(token)) for token in doc] for doc in docs_ls]

CPU times: user 21.6 s, sys: 248 ms, total: 21.8 s
Wall time: 24.2 s


In [ ]:
# VERFIY: functionality

lemmatizer.lemmatize('ran', wordnet.VERB)
lemmatizer.lemmatize('running', wordnet.VERB)
lemmatizer.lemmatize('run', wordnet.VERB)

'run'

'run'

'run'

In [ ]:
# VERIFY: Compare pre/post lemmatization

doc_no = -1
token_ct = 100

for i in range(token_ct):
  print(f'ORIGINAL: {docs_ls[doc_no][i]} --> LEMMA: {docs_lemma_ls[doc_no][i]}')


ORIGINAL: technology --> LEMMA: technology
ORIGINAL: 14344 --> LEMMA: 14344
ORIGINAL: ted --> LEMMA: ted
ORIGINAL: conferences --> LEMMA: conference
ORIGINAL: 1315 --> LEMMA: 1315
ORIGINAL: 42 --> LEMMA: 42
ORIGINAL: 8889 --> LEMMA: 8889
ORIGINAL: 142 --> LEMMA: 142
ORIGINAL: 248 --> LEMMA: 248
ORIGINAL: television --> LEMMA: television
ORIGINAL: 122 --> LEMMA: 122
ORIGINAL: 231 --> LEMMA: 231
ORIGINAL: therapy --> LEMMA: therapy
ORIGINAL: 57 --> LEMMA: 57
ORIGINAL: 56 --> LEMMA: 56
ORIGINAL: 97 --> LEMMA: 97
ORIGINAL: tiger --> LEMMA: tiger
ORIGINAL: moms --> LEMMA: mom
ORIGINAL: 216 --> LEMMA: 216
ORIGINAL: tightrope --> LEMMA: tightrope
ORIGINAL: walking --> LEMMA: walk
ORIGINAL: 168-171 --> LEMMA: 168-171
ORIGINAL: time --> LEMMA: time
ORIGINAL: 154 --> LEMMA: 154
ORIGINAL: top --> LEMMA: top
ORIGINAL: gun --> LEMMA: gun
ORIGINAL: 1986 --> LEMMA: 1986
ORIGINAL: 104 --> LEMMA: 104
ORIGINAL: tragedy --> LEMMA: tragedy
ORIGINAL: 126 --> LEMMA: 126
ORIGINAL: see --> LEMMA: see
ORIGINAL

We find bigrams in the documents. Bigrams are sets of two adjacent words.
Using bigrams we can get phrases like "machine_learning" in our output
(spaces are replaced with underscores); without bigrams we would only get
"machine" and "learning".

Note that in the code below, we find bigrams and then add them to the
original data, because we would like to keep the words "machine" and
"learning" as well as the bigram "machine_learning".

.. Important::
    Computing n-grams of large dataset can be very computationally
    and memory intensive.




### Identify Bi- and Tri-Grams

In [ ]:
import numpy as np
from gensim.models import Phrases
from gensim.models.phrases import Phraser

# If docs_lemma_ls is not defined, create a small example for demonstration:
try:
    docs_lemma_ls
except NameError:
    docs_lemma_ls = [['this', 'is', 'a', 'document'], ['another', 'document', 'example']]

# --- Bigram Extraction ---
# Create a Phrases model to find bigrams that appear at least 20 times.
bigram = Phrases(docs_lemma_ls, min_count=20)
bigram_phraser = Phraser(bigram)

for idx in range(len(docs_lemma_ls)):
    # Apply the bigram phraser to each document
    bigram_tokens = bigram_phraser[docs_lemma_ls[idx]]
    # Append only tokens that are identified as bigrams (contain an underscore)
    docs_lemma_ls[idx].extend([token for token in bigram_tokens if '_' in token])

# --- Trigram Extraction ---
# Build a Phrases model on the output of the bigram phraser
trigram = Phrases(bigram_phraser[docs_lemma_ls], min_count=20)
trigram_phraser = Phraser(trigram)

for idx in range(len(docs_lemma_ls)):
    # First apply the bigram phraser
    bigram_tokens = bigram_phraser[docs_lemma_ls[idx]]
    # Then apply the trigram phraser on the bigram output
    trigram_tokens = trigram_phraser[bigram_tokens]
    # Append only tokens that are trigrams (contain exactly two underscores)
    docs_lemma_ls[idx].extend([token for token in trigram_tokens if token.count('_') == 2])

# --- Verification with an Example ---
example_sentences = [
    ['this', 'is', 'a', 'sentence'],
    ['another', 'sentence'],
    ['yet', 'another', 'sentence']
]
example_bigram = Phrases(example_sentences, min_count=1, threshold=2, delimiter='_')
example_bigram_phraser = Phraser(example_bigram)
example_bigram_sentences = [example_bigram_phraser[sent] for sent in example_sentences]

print("Bigram results:", example_bigram_sentences)


**[SKIP TO NEXT SECTION]**

In [ ]:
for token in bigram[docs_lemma_ls[idx]]:
  if '_' in token:
    print(token)

shame_resilience
dare_greatly
shame_resilience
shame_resilience
shame_resilience
dare_greatly
shame_resilience
shame_resilience


We remove rare words and common words based on their *document frequency*.
Below we remove words that appear in less than 20 documents or in more than
50% of the documents. Consider trying to remove words only based on their
frequency, or maybe combining that with this approach.




In [ ]:
#@markdown Minimum number of documents a word must appear in: (default 20)

Min_No_Documents = 20 #@param {type:"slider", min:5, max:100, step:1}

#@markdown Max percent of documents a word can appear in: (default 0.50 or 50%)

Max_Percent_Documents = 0.5 #@param {type:"slider", min:0.05, max:0.9, step:0.01}


In [ ]:
# Remove rare and common tokens.
from gensim.corpora import Dictionary

# Create a dictionary representation of the documents.
dictionary = Dictionary(docs_lemma_ls)

dict_raw_len = len(dictionary)
print(f'Dictionary length (raw): {dict_raw_len}')

Dictionary length (raw): 18277


In [ ]:
# Filter out words that occur less than 20 documents, or more than 50% of the documents.
dictionary.filter_extremes(no_below=Min_No_Documents, no_above=Max_Percent_Documents)

dict_clean_len = len(dictionary)
print(f'Dictionary length (filtered): {dict_clean_len}')

Dictionary length (filtered): 1469


In [ ]:
print(f'{dict_raw_len-dict_clean_len} Words removed from {dict_raw_len} or\n {(((dict_raw_len-dict_clean_len)/dict_raw_len)*100):.1f}% removed')

16808 Words removed from 18277 or
 92.0% removed


Finally, we transform the documents to a vectorized form. We simply compute
the frequency of each word, including the bigrams.




In [ ]:
# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in docs_lemma_ls]

Let's see how many tokens and documents we have to train on.




In [ ]:
# Corpus as a list of documents, each a list of tokens identifyed by dictionary tuples

len(corpus)

748

In [ ]:
type(corpus[0])

list

In [ ]:
corpus[0][:10]

[(0, 13),
 (1, 2),
 (2, 1),
 (3, 2),
 (4, 1),
 (5, 1),
 (6, 2),
 (7, 1),
 (8, 1),
 (9, 1)]

In [ ]:
print('Number of unique tokens: %d' % len(dictionary))  # Orig 1864
print('Number of documents: %d' % len(corpus))          # Orig 1740

Number of unique tokens: 1469
Number of documents: 748


## Training

We are ready to train the LDA model. We will first discuss how to set some of
the training parameters.

First of all, the elephant in the room: **how many topics do I need?** There is
really no easy answer for this, it will depend on both your data and your
application. I have used 10 topics here because I wanted to have a few topics
that I could interpret and "label", and because that turned out to give me
reasonably good results. You might not need to interpret all your topics, so
you could use a large number of topics, for example 100.

``chunksize`` controls how many documents are processed at a time in the
training algorithm. Increasing chunksize will speed up training, at least as
long as the chunk of documents easily fit into memory. I've set ``chunksize =
2000``, which is more than the amount of documents, so I process all the
data in one go. **Chunksize can however influence the quality** of the model, as
discussed in Hoffman and co-authors [2], but the difference was not
substantial in this case.

``passes`` controls how often we train the model on the entire corpus.
Another word for passes might be "epochs". ``iterations`` is somewhat
technical, but essentially it controls how often we repeat a particular loop
over each document. It is important to set the number of "passes" and
"iterations" high enough.

I suggest the following way to choose iterations and passes. First, enable
logging (as described in many Gensim tutorials), and set ``eval_every = 1``
in ``LdaModel``. When training the model look for a line in the log that
looks something like this::

   2016-06-21 15:40:06,753 - gensim.models.ldamodel - DEBUG - 68/1566 documents converged within 400 iterations

If you set ``passes = 20`` you will see this line 20 times. Make sure that by
the final passes, most of the documents have converged. So you want to choose
both passes and iterations to be high enough for this to happen.

We set ``alpha = 'auto'`` and ``eta = 'auto'``. Again this is somewhat
technical, but essentially we are automatically learning two parameters in
the model that we usually would have to specify explicitly.




In [ ]:
#@markdown How many Topics do you want to find?

No_of_Topics = 12 #@param {type:"slider", min:2, max:200, step:1}

#@markdown Default 10-50 depending upon how large the text and diverse the vocabulary

In [ ]:
%%time

# NOTE: 2m01s @20220328 with Google Colab/CPU on Harry Potter
#       1m05s 20230401 at 10:37EST with Google Colab/CPU on JourneyToTheWest_4vol_2003_Chengen

# Train LDA model.
from gensim.models import LdaModel

# Set training parameters.
num_topics = No_of_Topics
chunksize = 2000
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

CPU times: user 17.9 s, sys: 156 ms, total: 18 s
Wall time: 19.4 s


We can compute the topic coherence of each topic. Below we display the
average topic coherence and print the topics in order of topic coherence.

Note that we use the **"Umass" topic coherence** measure here (see
:py:func:`gensim.models.ldamodel.LdaModel.top_topics`), Gensim has recently
obtained an implementation of the "AKSW" topic coherence measure (see
accompanying blog post, http://rare-technologies.com/what-is-topic-coherence/).

If you are familiar with the subject of the articles in this dataset, you can
see that the topics below make a lot of sense. However, they are not without
flaws. We can see that there is substantial overlap between some topics,
others are hard to interpret, and most of them have at least some terms that
seem out of place. If you were able to do better, feel free to share your
methods on the blog at http://rare-technologies.com/lda-training-tips/ !




In [ ]:
top_topics = model.top_topics(corpus) #, num_words=20)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.

from pprint import pprint
pprint(top_topics)

print('\n\n')
for i,atopic in enumerate(top_topics):
  print(f'Topic #{i}: coherence = {atopic[1]}')

avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('\nAverage topic coherence: %.4f.' % avg_topic_coherence)

[([(0.015638717, 'he'),
   (0.011964292, 'say'),
   (0.009098527, 'would'),
   (0.008542497, 'go'),
   (0.007464408, 'year'),
   (0.0064951987, 'back'),
   (0.006035088, 'feel'),
   (0.0057778955, 'something'),
   (0.005747761, 'me'),
   (0.005675216, 'look'),
   (0.005645289, 'she'),
   (0.005472184, 'give'),
   (0.0050233556, 'good'),
   (0.005008141, 'that'),
   (0.0048756986, 'when'),
   (0.004845416, 'could'),
   (0.0048253313, 'come'),
   (0.00476745, 'know'),
   (0.004694675, 'did not'),
   (0.004692456, 'never')],
  -0.9891825334312132),
 ([(0.012426528, 'you'),
   (0.008507714, 'something'),
   (0.0075715594, 'world'),
   (0.0075240764, 'know'),
   (0.0069036693, 'good'),
   (0.006883869, 'problem'),
   (0.006821034, 'that'),
   (0.0065370393, 'could'),
   (0.006455516, 'we'),
   (0.0061868257, 'would'),
   (0.0061767832, 'child'),
   (0.00586072, 'might'),
   (0.005833626, 'that is'),
   (0.005495437, 'god'),
   (0.005475915, 'u'),
   (0.005427464, 'what'),
   (0.005240328, '

# Visualize

---

### 1. The Intertopic Distance Map (Left Panel)

- **Topics as Circles:**  
  Each circle represents a topic. The area of the circle is roughly proportional to the topic's prevalence in your corpus (i.e., how many documents or how much text is assigned to that topic).

- **Distance Between Topics:**  
  The distance between circles reflects the similarity between topics. Topics that are closer together share more similar word distributions, whereas topics that are farther apart are more distinct. If circles overlap significantly, it suggests that those topics might be redundant or not well separated.

- **Axes:**  
  The axes are generated via a dimensionality reduction technique (often t-SNE or multidimensional scaling). Although the axes themselves do not have a direct interpretative meaning, they provide a spatial summary of topic similarities.

---

### 2. The Term Relevance Panel (Right Panel)

- **Top Terms:**  
  This panel shows a ranked list of terms (words) that are most relevant for the selected topic. The list is dynamic: clicking on a topic (a circle in the left panel) updates the list of words shown.

- **Relevance Metric (\(\lambda\)):**  
  pyLDAvis allows you to adjust a slider for the relevance metric \(\lambda\). This metric balances:
  - **Term Probability:** How often the word appears in the topic.
  - **Term Exclusivity:** How unique the word is to that topic compared to others.
  
  At $\lambda = 1$, words are ranked purely by their probability in the topic (frequent words). At lower \(\lambda\) values, more weight is given to words that are more exclusive to the topic. This balance helps in identifying words that not only are common within a topic but also distinguish it from others.

- **Bar Chart Interpretation:**  
  Next to each term, you typically see bars or numbers indicating:
  - The overall frequency of the term in the corpus.
  - The weight or importance of the term in the topic.
  
  These details help you assess whether a term is both representative and unique for a topic.

---

### 3. Interactive Exploration

- **Topic Selection:**  
  Clicking on any topic circle in the intertopic distance map highlights that topic and updates the list of top terms on the right panel. This allows you to drill down into each topic’s composition.

- **Adjusting $\lambda$:**  
  Experiment with the $\lambda$ slider to see how the ranking of terms changes. This helps to understand both the common and the more discriminative words in a topic. For instance, if certain terms disappear as you lower \(\lambda\), they might be common across multiple topics rather than unique to the one you’re examining.

---

### 4. Drawing Conclusions

- **Topic Coherence and Interpretability:**  
  Look at whether the top terms for a topic form a coherent theme or concept. A well-separated topic should have a set of terms that collectively make sense and align with a known theme or domain.

- **Overlap and Redundancy:**  
  If topics are overlapping considerably on the distance map, it may indicate that the model is producing redundant topics. This might be a cue to adjust the number of topics or other hyperparameters.

- **Model Tuning:**  
  Use the insights from the visualization to refine your model. For example, if some topics appear too general (with common words across topics), you might adjust the model’s hyperparameters (e.g., Dirichlet priors) or the preprocessing steps (e.g., stopword removal) to encourage more distinct topics.

---

### Summary

To interpret a pyLDAvis visualization effectively:
1. **Examine the intertopic distance map** to see the distinctiveness and prevalence of topics.
2. **Review the top terms list** for each topic, considering both their frequency and exclusivity.
3. **Utilize interactive features** (clicking topics and adjusting \(\lambda\)) to explore how words contribute to each topic.
4. **Refine your model** based on observed overlaps or unclear topics to achieve better topic coherence and separation.

This structured interpretation can help you not only understand the output of your LDA model but also guide iterative improvements for more meaningful topic discovery.

In the context of topic modeling visualization with pyLDAvis, **λ (lambda)** is a tuning parameter used to balance two aspects of term importance when ranking words for each topic. The relevance of a term \( w \) in a topic \( t \) is defined as:

$$
r(w, t \mid \lambda) = \lambda \log p(w \mid t) + (1-\lambda) \log \frac{p(w \mid t)}{p(w)}
$$

Where:
- \( p(w \mid t) \) is the probability of the term \( w \) in the topic \( t \).
- \( p(w) \) is the overall probability of the term \( w \) in the entire corpus.

### What Does This Mean?

- **When \(\lambda = 1\):**  
  The ranking is based solely on the term's probability within the topic (\( p(w \mid t) \)). In this case, the most frequent words in the topic are shown, but they might also be common across the entire corpus, which could reduce the distinctiveness of the topic.

- **When \(\lambda = 0\):**  
  The ranking is based solely on the term's lift, \( \frac{p(w \mid t)}{p(w)} \), which measures how exclusive a word is to that topic relative to its overall frequency. This helps surface words that are more unique to the topic, even if they are not the most frequent.

- **Intermediate Values (e.g., \(\lambda = 0.6\)):**  
  Provide a balanced view by considering both frequency and exclusivity, often leading to a more interpretable set of top terms for each topic.

### Why Is Lambda Useful?

- **Interactivity:**  
  In pyLDAvis, you can interactively adjust the \(\lambda\) slider. This allows you to see how the list of top terms for a topic changes based on the balance between commonality and uniqueness.
  
- **Enhanced Interpretability:**  
  By tuning \(\lambda\), you can identify words that are not only frequent in the topic but also distinctive compared to the overall corpus. This improves the semantic clarity of the topics.

In summary, \(\lambda\) is a critical parameter for refining the interpretability of your LDA model's output in pyLDAvis, enabling you to explore different aspects of term relevance within topics.

In [ ]:
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

In [ ]:
# vis_data = gensimvis.prepare(lda, corpus, dictionary)
vis_data = gensimvis.prepare(model, corpus, dictionary)
pyLDAvis.display(vis_data)

## Things to experiment with

* ``no_above`` and ``no_below`` parameters in ``filter_extremes`` method.
* Adding trigrams or even higher order n-grams.
* Consider whether using a hold-out set or cross-validation is the way to go for you.
* Try other datasets.

## Where to go from here

* Check out a RaRe blog post on the AKSW topic coherence measure (http://rare-technologies.com/what-is-topic-coherence/).
* pyLDAvis (https://pyldavis.readthedocs.io/en/latest/index.html).
* Read some more Gensim tutorials (https://github.com/RaRe-Technologies/gensim/blob/develop/tutorials.md#tutorials).
* If you haven't already, read [1] and [2] (see references).

## References

1. "Latent Dirichlet Allocation", Blei et al. 2003.
2. "Online Learning for Latent Dirichlet Allocation", Hoffman et al. 2010.


